In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, SimpleRNN, LSTM
from tensorflow.keras.utils import to_categorical
import joblib
import matplotlib.pyplot as plt

In [2]:
# Load preprocessed data
X = np.load("X.npy")
y = np.load("y.npy")

In [3]:
# Convert target to categorical (one-hot encoding)
y = to_categorical(y)

In [4]:
X = X.reshape(X.shape[0], X.shape[1], 1)

In [5]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Define RNN model
model = Sequential([
    SimpleRNN(64, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], 1)),  # First RNN layer
    Dropout(0.3),

    SimpleRNN(32, activation='relu', return_sequences=False),  # Second RNN layer
    Dropout(0.3),

    Dense(128, activation='relu'),
    Dropout(0.3),

    Dense(y_train.shape[1], activation='softmax')  # Output layer
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)               │ (None, 14, 64)              │           4,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 14, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ (None, 32)                  │           3,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │           4,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,068 (47.14 KB)

 Trainable params: 12,068 (47.14 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Train the model
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=18,
                    batch_size=32)

Epoch 1/18
100/100 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - accuracy: 0.4489 - loss: 1.1745 - val_accuracy: 0.7625 - val_loss: 0.5533
Epoch 2/18
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6967 - loss: 0.6482 - val_accuracy: 0.8100 - val_loss: 0.4313
Epoch 3/18
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7766 - loss: 0.5113 - val_accuracy: 0.8550 - val_loss: 0.3418
Epoch 4/18
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8228 - loss: 0.4331 - val_accuracy: 0.9350 - val_loss: 0.2103
Epoch 5/18
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8602 - loss: 0.3262 - val_accuracy: 0.9563 - val_loss: 0.1339
Epoch 6/18
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9051 - loss: 0.2536 - val_accuracy: 0.9775 - val_loss: 0.0909
Epoch 7/18
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9343 - loss: 0.1842 - val_accuracy: 0.9663 - val_loss: 0.0972
Epoch 8/18
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9397 - loss: 0.1819 - val_accur

In [16]:
# Save the model
model.save("rnn_network_attack_classifier.h5")
print("Model saved as cnn_network_attack_classifier.h5")

Model saved as cnn_network_attack_classifier.h5


In [17]:
# Save training history
np.save("rnn_training_history.npy", history.history)
print("Training history saved as training_history.npy")

Training history saved as training_history.npy
